In [70]:
# Algoritmo base
# https://www.cs.utexas.edu/~flame/Notes/NotesOnCholReal.pdf

# Alterações feitas:

# A matriz L = zero(A) está sendo criada fora da função
# As alterações estão sendo feitas na própria matriz A
# Utilizar .= e .-=, operações in-place, otimizaram a alocação

In [71]:
using(LinearAlgebra)

In [72]:
##### Nível 2

# Algoritmo do livro (applied numerical linear algebra)
function  Cholesky_factorize(A, L)
    # L: matriz de zeros que será alterada, mesmo tamanho de A
    
    for j in 1:size(A, 1)

        L[j, j] = (A[j, j] - sum(L[j, 1:j-1].^2))^(.5)

        for i in (j+1):size(A, 1)
        
            L[i, j] = (A[i, j] - dot(L[i, 1:j-1], L[j, 1:j-1]))/L[j, j]

        end

    end

    return L
end

Cholesky_factorize (generic function with 1 method)

In [73]:
### Nível 3 ######

function Cholesky_by_blocks(A, block_size)

    n = size(A, 1)
    
    for j in 1:block_size:n

        b = min(n - j + 1, block_size)
        # Realizando as alterações na própria matriz A, poupa alocações
        A[j:(j+b-1), j:(j+b-1)] .= cholesky(A[j:(j + b - 1), j:(j + b - 1)]).L
        A[(j+b):n, j:( j + b - 1)] = A[(j+b):n, j:(j + b -1)]*inv(A[j:(j + b - 1), j:(j + b - 1)]')
        A[(j+b):n, (j + b):n] .-= A[(j+b):n, j:(j + b - 1)]*A[(j+b):n, j:(j + b - 1)]'

    end

    return tril(A)
end

Cholesky_by_blocks (generic function with 2 methods)

In [5]:
# Testando com Cholesky nível 2

n = 5
X = randn(n, n)

A = X*X'

L = Cholesky_factorize(A, zero(A))

display(A)

display(L)

display(L*L')

display(A - L*L')

display(norm(A - L*L'))

5×5 Matrix{Float64}:
  3.83913    2.7338    -0.756689   0.921091   3.58136
  2.7338     6.71768   -2.54278   -1.19071   -0.991081
 -0.756689  -2.54278    3.98064   -2.50242    2.20876
  0.921091  -1.19071   -2.50242    4.29031    0.767046
  3.58136   -0.991081   2.20876    0.767046   7.30593

5×5 Matrix{Float64}:
  1.95937    0.0        0.0       0.0        0.0
  1.39524    2.18426    0.0       0.0        0.0
 -0.38619   -0.91745    1.7291    0.0        0.0
  0.470095  -0.845417  -1.79082   0.384157   0.0
  1.82781   -1.62129    0.825391  0.0397057  0.808447

5×5 Matrix{Float64}:
  3.83913    2.7338    -0.756689   0.921091   3.58136
  2.7338     6.71768   -2.54278   -1.19071   -0.991081
 -0.756689  -2.54278    3.98064   -2.50242    2.20876
  0.921091  -1.19071   -2.50242    4.29031    0.767046
  3.58136   -0.991081   2.20876    0.767046   7.30593

5×5 Matrix{Float64}:
 0.0   0.0          0.0  0.0   0.0
 0.0  -8.88178e-16  0.0  0.0   0.0
 0.0   0.0          0.0  0.0   0.0
 0.0   0.0          0.0  0.0   0.0
 0.0   0.0          0.0  0.0  -8.88178e-16

1.2560739669470201e-15

In [75]:
n = 3000
X = randn(n, n)
size_block = 2048
A = X*X'
B = copy(A)

display(isposdef(A))

@time Cholesky_factorize(A, zero(A))
# Esta função está alterando diretamente a matriz A, por isso fiz copias de A
@time Cholesky_by_blocks(A, size_block)
@time cholesky(B)

true

156.185266 seconds (25.55 M allocations: 67.905 GiB, 13.80% gc time)
  6.326036 seconds (57 allocations: 336.568 MiB, 0.70% gc time)
  2.793130 seconds (4 allocations: 68.665 MiB)


Cholesky{Float64, Matrix{Float64}}
U factor:
3000×3000 UpperTriangular{Float64, Matrix{Float64}}:
 54.0193   1.30559  -0.942096  …  -1.11167     0.28631   -0.141234
   ⋅      55.7505   -0.562999     -0.0345803  -1.62654   -0.220445
   ⋅        ⋅       53.5039       -1.55653     0.883094  -0.021005
   ⋅        ⋅         ⋅            0.193384    1.85367    0.31911
   ⋅        ⋅         ⋅           -0.75952    -0.679971   0.616948
   ⋅        ⋅         ⋅        …  -0.0770728  -0.400883  -0.76984
   ⋅        ⋅         ⋅           -0.221675    1.44972   -0.821316
   ⋅        ⋅         ⋅           -0.347639   -0.497066   1.37497
   ⋅        ⋅         ⋅           -0.289985   -1.45553   -0.801428
   ⋅        ⋅         ⋅            1.11858     1.51475   -1.21315
   ⋅        ⋅         ⋅        …  -1.11511     0.964191   0.289936
   ⋅        ⋅         ⋅            1.3251      0.741173  -0.37182
   ⋅        ⋅         ⋅            0.899014    0.680929   0.271023
  ⋮                            ⋱    

In [68]:
n = 1500
X = randn(n, n)
size_block = 1024

A = X*X'
B = copy(A)

L = Cholesky_factorize(A, zero(A))
C = Cholesky_by_blocks(A, size_block)
F = cholesky(B)


display(norm(B - L*L'))
display(norm(B - C*C'))
display(norm(B - F.L*F.L'))

2.3394614212697704e-11

2.801978156752108e-11

2.0063531514909436e-11